# DPPL Algorithm
## Isaac Schaal

A Python implementation of the DPPL Algorithm, based on the version presented in Russel & Norvig. 

In [1]:
import numpy as np
import copy
import random

class Literal:
    # Initiates the object
    def __init__(self, name, sign = True):
        self.name = name
        self.sign = sign
    
    # The neg operator returns a new object
    # with the same name but opposite sign
    def __neg__(self):
        new_sign = not self.sign
        return Literal(self.name, new_sign)

def DPLLSatisfiable(KB):
    """ The wrapper function for the recursive
    DPPL algorithm. Takes as input a KB of the form 
    
    KB = [{A,B},{A, -C},{-A,B,D}]
    
    (which represents (A ∨ B) ∧ (A ∨ ¬C) ∧ (¬A ∨ B ∨ D))
    
    The function should be called like :
    (satisfiable, model) = DPLLSatisfiable(KB)
    
    Satisfiable returns True if the model is satisfiable, 
    and model returns a (note not the only) model that satisfies
    the KB
    
    Satifiable returns False if the model is unsatisfiable,
    and model returns None
    
    """
    # good_model is a global variable that 
    # keeps the successful model when all 
    # clauses are found to be true
    global good_model
    # Its initiated to None in case there is
    # no model that works
    good_model = None
    
    # Create a list of symbols
    lis = []
    for clause in KB:
        for literal in clause:
            lis.append((literal.name))
    symbols = list(np.unique(lis))
    
    # Use that list to create a model
    # where all values are free
    model = {} # Values can be 'true', 'false', or 'free' 
    for i in symbols:
        model[i] = 'free'
        
    # Start the DPLL recursion
    satisfiable = DPLL(KB,model)
    
    return (satisfiable , good_model)


def DPLL(KB, model):  
    
    """
    This function is the main recursive function
    of the algorithm. 
    
    It selects p which is the free symbol that
    will be tried as both 'true' and 'false' in the next
    recursion. It creates new models (one with each option)
    and then recursively calls itself on both new models.
    
    Note that the exiting the recursion comes from the
    model_eval() function
    
    """  
    
    # Check if either all models are true
    # or at least one model is false
    cur_state,t_vals = model_eval(KB, model)
    
    # If all models are True, exit the
    # recursion as True
    # If a model is False, exit the 
    # recurison as False
    if cur_state != None:
        return cur_state
    
    ##### Attempt the Pure Symbol Heuristic #####
    p, value = Find_Pure_Symbol(KB, t_vals, model)
    if p != None:
        pure_symbol_model = copy.deepcopy(model)
        pure_symbol_model[p] = value
        return DPLL(KB,pure_symbol_model)
    
    ##### Attempt the Unit Clause Heuristic #####
    p, value = Find_Unit_Clause(KB, t_vals, model)
    if p != None:
        unit_clause_model = copy.deepcopy(model)
        unit_clause_model[p] = value
        return DPLL(KB,unit_clause_model)
        
        
    ##### Choose P according to 'Degree Heuristic' #####
    # Get a list of each symbol and how many times it occurs#
    lis = []
    for clause in KB:
        for literal in clause:
            lis.append((literal.name))
    symbols = np.unique(lis, return_counts = True)
    # symbols[0] is the symbols and symbols[1] is the counts
    
    # Sort the literals based on their counts
    literal_list = [x for _,x in sorted(zip(symbols[1],symbols[0]))]
    
    # Iterate throught the sorted list and 
    # choose the first that is free
    for k in literal_list:
        if model[k] == 'free':
            p = k
    
    # Set P to 'true' and 'false'
    # and recursively run DPLL again
    # on the new models
    t_model = copy.deepcopy(model)
    t_model[p] = 'true'
    t_state = DPLL(KB, t_model)
    
    f_model = copy.deepcopy(model)
    f_model[p] = 'false'
    f_state = DPLL(KB, f_model)
    
    # If either is true, move up the recursion
    # succesfully, and move up as False if not
    if t_state == True or f_state == True:
        return True
    else:
        return False
    
    
def model_eval(KB, model):
    """
    This function takes a KB and a model
    and determines if the either all clauses are 
    True, one clause is False, or neither of the above.
    
    It is ran at the beggining of every call to DPPL().
    
    If all clauses are True, a model that satisfies the KB
    has been found, its saved to good_model, and we will move 
    up the recursion.
    
    If all clauses are False, the model can't satisfy the KB
    and we will move up the recursion.
    
    If neither of the above are true, we will continue down
    the recursion.
    """
    global good_model
    
    # create arrays for recording if
    # clauses are true
    t_vals = []
    # Iterate through all clauses
    for clause in KB:
        t = False
        f = True
        #Iterate through all literals in the clause
        for literal in clause:
            # if any literal is true and its sign is 
            # true or is false and its sign is false,
            # then the clause is true
            if model[literal.name] == 'true':
                if literal.sign == True:
                    t = True
                    f = False
                    break                    
            elif model[literal.name] == 'false':
                if literal.sign == False:
                    t = True
                    f= False
                    break     
            # The clause is not false if there is
            # a free literal
            elif model[literal.name] == 'free':
                    f = False
                    
            # If no literals are free and none
            # of the literals are true (including 
            # their sign), then f will remain True
            # as the clause is false
            
        # If there is a False clause, return False
        if f == True:
            return False, t_vals
        
        #Record if the clause is true
        t_vals.append(t)

    # If there are all True clauses, return True
    if sum(t_vals) == len(t_vals):
        # and save the model to the global good_model
        good_model = copy.deepcopy(model)
        return True, t_vals
    # Else return none
    else:
        return None, t_vals
    
    
def Find_Pure_Symbol(KB, t_vals, model):
    """
    This function finds a pure symbol if there is one.
    A pure symbol appears with the same sign in all clauses
    and thus can be set to that sign. Note that if a clause
    is already true, than any symbol in that clause don't 
    count towards a symbol being a pure symbol. 
    
    If pure symbols can be found, it returns the pure 
    symbol that occurs the most in the clauses and 
    the appropriate value, and if  there is no pure symbol
    it returns None

    """
    
    # creates a set of all symbols
    # that occur as positive and another
    # of all symbols that occur as negative
    pos_symbols = set()
    neg_symbols = set()
    for i,clause in enumerate(KB):
    # for all clauses that are not already True
        if t_vals[i] != True:
            for literal in clause:
                if literal.sign ==True:
                    pos_symbols.add(literal.name)
                elif literal.sign == False:
                    neg_symbols.add(literal.name)
                    
    # Find pure symbols
    pure_symbol_signs = {}
    pure_symbols = []
    
    for symbol in pos_symbols:
        if symbol not in neg_symbols:
            # checks that the symbol is free
            if model[symbol] == 'free':
                pure_symbols.append(symbol)
                pure_symbol_signs[symbol] = 'true'
                
    for symbol in neg_symbols:
        if symbol not in pos_symbols:
            # checks that the symbol is free
            if model[symbol] == 'free':
                pure_symbols.append(symbol)
                pure_symbol_signs[symbol] = 'false'
                
    # finds the count of each pure symbol in the non
    # true clauses
    pure_symbol_counts = []
    for pure_symbol in pure_symbols:
        count = 0
        for i,clause in enumerate(KB):
        # for all clauses that are not already True
            if t_vals[i] != True:
                for literal in clause:
                    if literal.name == pure_symbol:
                        count +=1
        pure_symbol_counts.append(count)
    
    # if there is no pure symbol, return None
    if len(pure_symbols) == 0:
        return None, None
    # else, return the pure symbol that occurs the most
    # and its sign
    else:
        return_symbol = pure_symbols[pure_symbol_counts.index(max(pure_symbol_counts))]
        return_sign = pure_symbol_signs[return_symbol]
        return return_symbol, return_sign


def Find_Unit_Clause(KB, t_vals, model):
    """
    This function returns a unit clause, if there is one.
    A unit clause is a clause with only one literal or a clause 
    where all other literals are false. We thus know that we must 
    set the remaining (or only, if theres one) literal to the
    appropriate value.
    
    Returns the symbol and the value if there is a unit clause.
    """
    
    p,value = None, None
    
    for i,clause in enumerate(KB):
    # for all clauses that are not already True
        if t_vals[i] != True:
            # Unit clauses that are of length one
            if len(clause) == 1:
                p_possible = list(clause)[0]
                if model[p_possible.name] == 'free':
                    p = p_possible
                    value = str(p.sign).lower()
                    p = p.name
                    break
            else:
            # or unit clauses where all but one are false
                false_list = []
                # record the literals that are false
                for literal in clause:
                    is_false = 0
                    if model[literal.name] == 'true' and literal.sign == False:
                        is_false = 1
                    elif model[literal.name] == 'false' and literal.sign == True:
                        is_false = 1
                    false_list.append(is_false)
                # if theres only one non false, its a unit clause
                if sum(false_list) == len(false_list) -1:
                    index = false_list.index(0)
                    p = list(clause)[index]
                    value = str(p.sign).lower()
                    p = p.name
                    break
    return p, value

We will test the algorithm on the KB provided in Exercise 7.20 of Russell & Norvig. We have 6 sentences that we must convert into Conjunctive Normal Form

1. A ⇔ (B ∨ E)
2. E ⇒ D
3. C ∧ F ⇒ ¬B
4. E ⇒ B
5. B ⇒ F
6. B ⇒ C

The conversions are as follows

1.  A ⇔ (B ∨ E)  

 (A ⇒ (B ∨ E)) ∧ ((B ∨ E)⇒A) - Biconditional Elimination

 (¬A ∨ B ∨ E) ∧ (¬(B ∨ E) ∨ A) - Implication Elimination

 (¬A ∨ B ∨ E) ∧ ((¬B ∧ ¬E) ∨ A) - De Morgans

 (¬A ∨ B ∨ E) ∧ (¬B ∨ A) ∧ (¬E ∨ A) - Distributive Law
 


2. E ⇒ D

 (¬E ∨ D) - Implication Elimination
 
 
 
3. C ∧ F ⇒ ¬B

 ¬(C ∧ F) ∨ ¬B - Implication Elimination
 
 (¬C ∨ ¬F ∨ ¬B) - De Morgans
  
 
4. E ⇒ B

  (¬E ∨ B) - Implication Elimination
  
 
5. B ⇒ F

  (¬B ∨ F) - Implication Elimination
  
  
6. B ⇒ C

  (¬B ∨ C) - Implication Elimination
  

and we can write this KB in Python using our `Literal` class as follows:

In [3]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A,B,E},
      {-B,A},
      {-E,A},
      {-E,D},
      {-C,-F,-B},
      {-E,B},
      {-B,F},
      {-B,C}]

# and to test the KB
(satisfiable, model) = DPLLSatisfiable(KB)
print (satisfiable)
print (model)

True
{'A': 'false', 'B': 'false', 'C': 'free', 'D': 'true', 'E': 'false', 'F': 'free'}
